# Plan of Action

**Our age prediction CNN model shall be defined and trained by**:
1. Importing **training and test datasets** from Google Drive Input Sub-folder
2. **Training dataset is already augmented** and has 234,000 images
3. **Greyscaling images** instead of using RGB color images
4. Defining our intuitively **distributed classes of age-ranges**
5. Using **60 epochs** on our **optimized CNN Architecture**, comprising of:
    - an input *Conv2D* layer (with 32 filters) paired with an *AveragePooling2D* layer,
    - 3 pairs of *Conv2D* (with 64, 128 & 256 filters) and *AveragePooling2D* layers,
    - a *GlobalAveragePooling2D* layer,
    - 1 *Dense* layer with 132 nodes, and
    - an output *Dense* layer with 7 nodes.

# Mount Google Drive & Imports

In [ ]:
#@title Mount Google Drive {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torchvision import datasets, transforms

import tensorflow as tf

# Setting random seeds to reduce the amount of randomness in the neural net weights and results
# The results may still not be exactly reproducible

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#@title Check for GPU

# Testing to ensure GPU is being utilized
# Ensure that the Runtime Type for this notebook is set to GPU
# If a GPU device is not found, change the runtime type under: Runtime>> Change runtime type>> Hardware accelerator>> GPU
# and run the notebook from the beginning again.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Data Preparation

## Import Dataset

In [ ]:
input_path = "/content/drive/MyDrive/DeepLearning_2023/Final Project/Data/"
# List all files in the folder
files = os.listdir(input_path+'UTKFace/')

# Initialize an empty list to store the image paths
image_paths = []
labels =[]
genderlist=[]
racelist = []
# Iterate over the files in the folder

for file in files:
  # Check if the file is an image
  if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
    # Construct the full path to the image file
    age = file.split("_")[0]
    gender = file.split("_")[1]
    race = file.split("_")[2]
    image_path = os.path.join(input_path+'UTKFace/', file)

    # Append the image path to the list
    image_paths.append(image_path)
    labels.append(age)
    genderlist.append(gender)
    racelist.append(race)


# Create a DataFrame with the image paths
df = pd.DataFrame({'image_path': image_paths,"age":labels,"gender":gender,"race":race})
df = df.astype({'age':float})




In [ ]:

display(df)

,image_path,age,gender,race
0,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
1,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
2,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
3,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
4,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
...,...,...,...,...
23703,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2
23704,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2
23705,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2
23706,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2


## Know your Dataset

In [ ]:
df['age'].describe()

count    23708.000000
mean        33.303484
std         19.886112
min          1.000000
25%         23.000000
50%         29.000000
75%         45.000000
max        116.000000
Name: age, dtype: float64

In [ ]:
df = df[df['age']<=100]

In [ ]:

import plotly.express as px
fig = px.histogram(df, x="age", nbins=100)
fig.update_layout(title_text='Age distribution')
fig.show()


## Ajust dataset

In [ ]:
print(df['age'].value_counts(),"\n")
#downsampling
df = df.drop(df[df['age'] == 26].sample(frac=.4).index)
print(df['age'].value_counts(),"\n")

26.0    2197
1.0     1123
28.0     918
35.0     880
24.0     859
        ... 
87.0      10
99.0       9
95.0       9
93.0       5
91.0       2
Name: age, Length: 97, dtype: int64 

26.0    1318
1.0     1123
28.0     918
35.0     880
24.0     859
        ... 
87.0      10
99.0       9
95.0       9
93.0       5
91.0       2
Name: age, Length: 97, dtype: int64 



In [ ]:
display(df)

,image_path,age,gender,race
0,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
1,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
2,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
3,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
4,/content/drive/MyDrive/DeepLearning_2023/Final...,56.0,0,2
...,...,...,...,...
23703,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2
23704,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2
23705,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2
23706,/content/drive/MyDrive/DeepLearning_2023/Final...,1.0,0,2


In [ ]:
fig = px.histogram(df, x="age", nbins=100)
fig.update_layout(title_text='Age distribution')
fig.show()

## Divide into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['image_path'], df['age'],test_size = 0.15,random_state = 42, stratify=df['age'])

## Dataloader


In [ ]:
# Converting the filenames and target class labels into lists for augmented train and test datasets.
train_filenames_list = list(x_train)
train_labels_list = list(y_train)

test_filenames_list = list(x_test)
test_labels_list = list(y_test)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as tf
import matplotlib.pyplot as plt
from PIL import Image
import scipy.io as sio
from google.colab import drive
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize the image to a specific size
    transforms.CenterCrop(150),
    transforms.ToTensor()  # Convert image to tensor
])


# Create the custom dataset
train_dataset = CustomDataset(train_filenames_list, train_labels_list, transform=transform)

# Create the dataloader
train_loader = DataLoader(train_dataset, batch_size=2000, shuffle=False) #PROVAR AUGMENTAR BACTHSIZE

# Create the custom dataset
test_dataset = CustomDataset(test_filenames_list, test_labels_list, transform=transform)

# Create the dataloader
test_loader = DataLoader(test_dataset, batch_size=2000, shuffle=False) #PROVAR AUGMENTAR BACTHSIZE


# CNN Architecture

In [ ]:
# source:
class MeanVarianceLoss(nn.Module):

    def __init__(self, lambda_1, lambda_2, start_age, end_age):
        super().__init__()
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.start_age = start_age
        self.end_age = end_age

    def forward(self, input, target):

        N = input.size()[0]
        target = target.type(torch.FloatTensor).cuda()
        m = nn.Softmax(dim=0)
        p = m(input)
        print(p)
        # mean loss
        a = torch.arange(self.start_age, self.end_age + 1, dtype=torch.float32).cuda()

        print(p.shape, a.shape)

        mean = torch.squeeze((p * a).sum(1, keepdim=True), dim=1) #### dimensions of a and p no quadren
        mse = (mean - target)**2
        mean_loss = mse.mean() / 2.0

        # variance loss
        b = (a[None, :] - mean[:, None])**2
        variance_loss = (p * b).sum(1, keepdim=True).mean()

        return self.lambda_1 * mean_loss, self.lambda_2 * variance_loss

In [ ]:
results_path = '/content/drive/MyDrive/DeepLearning_2023/Final Project/dl_nmm/1.1_age_input_output/output/'

def train(CNN, train_loader, test_loader, optimizer, num_epochs=5, model_name='model.ckpt', device='cpu', accumulation_steps=10):
    CNN.train() # Set the model in train mode
    total_step = len(train_loader)
    losses_list = []
    train_mae_list = []  # List to store training MAE
    test_losses_list = []
    test_mae_list = []  # List to store test MAE
    clip = 2

    criterion = nn.MSELoss()

    # Iterate over epochs
    for epoch in range(num_epochs):
        loss_avg = 0
        train_mae_avg = 0  # Average training MAE for the epoch
        nBatches = 0
        accumulation_loss = 0

        # Iterate over batches
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.type(torch.LongTensor).to(device)

            # Forward pass
            outputs = CNN(images)

            loss = criterion(outputs.squeeze().float(), labels.float())

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(CNN.parameters(), clip)

            # Accumulate gradients
            accumulation_loss += loss.item()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                loss_avg += accumulation_loss / accumulation_steps
                accumulation_loss = 0
                nBatches += 1

                if (i + 1) % 100 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                        epoch + 1, num_epochs, i + 1, total_step, loss_avg / nBatches))

        # Perform the final update if there are remaining accumulated gradients
        if accumulation_loss > 0:
            optimizer.step()
            optimizer.zero_grad()
            loss_avg += accumulation_loss / accumulation_steps
            nBatches += 1

        # Calculate MAE for training set
        train_mae = calculate_mae(CNN, train_loader, device)
        train_mae_avg = train_mae / len(train_loader)
        train_mae_list.append(train_mae_avg)

        # Perform testing
        loss_test, test_mae = test(CNN, test_loader, criterion)
        test_losses_list.append(loss_test)
        test_mae_list.append(test_mae)

        losses_list.append(loss_avg / nBatches)
        torch.save(CNN.state_dict(), results_path + '/' + model_name)

        print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Train MAE: {:.4f}, Test Loss: {:.4f}, Test MAE: {:.4f}'.format(
            epoch + 1, num_epochs, i + 1, total_step, loss_avg / nBatches, train_mae_avg, loss_test, test_mae))

    return losses_list, train_mae_list, test_losses_list, test_mae_list


def calculate_mae(CNN, data_loader, device):
    with torch.no_grad():
        total_mae = 0
        total_samples = 0

        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Get network predictions
            outputs = CNN(images)

            # Calculate MAE
            mae = torch.mean(torch.abs(outputs.squeeze() - labels.float()))
            total_mae += mae.item() * labels.size(0)  # Accumulate MAE
            total_samples += labels.size(0)  # Accumulate total samples

        avg_mae = total_mae / total_samples

    return avg_mae


def test(CNN, test_loader, criterion):
    with torch.no_grad():
        difference = 0
        correct = 0
        total = 0
        total_loss = 0
        total_mae = 0  # Initialize total MAE

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Get network predictions
            outputs = CNN(images)
            loss = criterion(outputs.squeeze().float(), labels.float())

            # Calculate Mean Absolute Error (MAE)
            mae = torch.mean(torch.abs(outputs.squeeze() - labels.float()))
            total_mae += mae.item()  # Accumulate the MAE

            # Compare with the ground-truth
            total_loss += loss

        avg_loss = total_loss / len(test_loader)
        avg_mae = total_mae / len(test_loader)

        return avg_loss, avg_mae


In [ ]:
import torch.nn as nn

class AgeEstimationModel(nn.Module):
    def __init__(self):
        super(AgeEstimationModel, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3,  padding=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3,  padding=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(32,64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.4),
            nn.Linear(64,128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.4),
            nn.Linear(128,128),
            nn.ReLU(inplace=True),
            nn.Linear(128,1),
            )
    def forward(self, x):

      x = self.conv1(x)

      return x


In [ ]:
#Train MyModel
age_model = AgeEstimationModel()
#ExactAgeEstimator_50
#Initialize optimizer
learning_rate = .005
optimizer = torch.optim.Adam(age_model.parameters(), lr=learning_rate, weight_decay=0.09)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
age_model = age_model.to(device)
losses_list, train_mae_list, test_losses_list, test_mae_list= train(age_model, train_loader, test_loader, optimizer, num_epochs=20, model_name='ExactAgeEstimator_30_mae.ckpt', device=device)


Epoch [1/20], Step [10/10], Train Loss: 1531.7647, Train MAE: 3.3482, Test Loss: 1516.2961, Test MAE: 33.4000
Epoch [2/20], Step [10/10], Train Loss: 1525.3438, Train MAE: 3.3394, Test Loss: 1510.4540, Test MAE: 33.3119
Epoch [3/20], Step [10/10], Train Loss: 1519.4596, Train MAE: 3.3297, Test Loss: 1504.0757, Test MAE: 33.2154
Epoch [4/20], Step [10/10], Train Loss: 1513.0071, Train MAE: 3.3177, Test Loss: 1495.9972, Test MAE: 33.0946
Epoch [5/20], Step [10/10], Train Loss: 1505.0204, Train MAE: 3.3013, Test Loss: 1485.2893, Test MAE: 32.9308


KeyboardInterrupt: ignored

In [ ]:
print(torch.Tensor(test_losses_list).numpy())

In [ ]:
plt.plot(losses_list,label="train")
plt.plot(torch.Tensor(test_losses_list).numpy(),label="test")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss model')
plt.savefig('/content/drive/MyDrive/DeepLearning_2023/Final Project/plots/exactage_2_50_epoch_hub.png')
plt.show()

# Checking Model Performance

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


model.load_state_dict(torch.load(results_path + 'ExactAgeEstimator_50_hub.ckpt'))
model.cuda()
criterion = nn.MSELoss()
labels_list = []
outputs_list = []
for images, labels in test_loader:
  images = images.to(device)
  labels = labels.to(device)
  # get network predictions
  outputs = model(images)
  outputs_list.append(outputs)
  labels_list.append(labels)
  loss = criterion(outputs.squeeze().float(), labels.float())
  # compare with the ground-truth

In [ ]:
print(labels_list)

In [ ]:
list_ = []
for element in outputs_list:
  for element2 in torch.Tensor(element).squeeze():
      list.append(torch.Tensor(element2).cpu().detach().numpy())

In [ ]:
print(list)


In [ ]:
y_pred = []
y_truth = []
for images, labels in test_loader:
  images = images.to(device)
  labels = labels.to(device)
  # get network predictions
  outputs = model(images)
  for idx,label in enumerate(labels):
    y_pred.append(outputs[idx].item())
    y_truth.append(label.item())


In [ ]:
plt.scatter(y_truth, y_pred)
plt.xlabel('y_truth')
plt.ylabel('y_pred')
plt.show()